# 1. Set up the notebook

Import all the modules we need.

In [ ]:
import numpy as np
import sympy as sym
import matplotlib.pyplot as plt

# 2. Show how to take cross products

### Taking cross products with numpy

Suppose we want to take the cross product
$ w \times v $
where $w$ and $v$ are described in coordinates as follows:

In [ ]:
w_inB = np.array([1., 2., 3.])
v_inB = np.array([4., 5., 6.])

The easiest way to do this is with [numpy.cross](https://numpy.org/doc/stable/reference/generated/numpy.cross.html):

In [ ]:
print(np.cross(w_inB, v_inB))

An alternative is to use the wedge operator. If

$$ w^B = \begin{bmatrix} a \\ b \\ c \end{bmatrix} $$

then

$$ \widehat{w^B} = \begin{bmatrix} 0 & -c & b \\ c & 0 & -a \\ -b & a & 0 \end{bmatrix} $$

and the cross product $w \times v$ in the coordinates of frame $B$ can be found by matrix multiplication:

$$ \left( w \times v \right)^B = \widehat{w^B} v^b. $$

The "wedge operator" is the mapping from the column vector $w^B$ to the skew-symmetric matrix $\widehat{w^B}$. Here is a function that implements this operator:

In [ ]:
def wedge_numpy(v):
    """
    Returns a skew-symmetric matrix M that can be used to implement
    the cross product "v x w" as the matrix product "M @ w" - assumes
    that v has shape (3, )
    """
    return np.array([[0., -v[2], v[1]], [v[2], 0., -v[0]], [-v[1], v[0], 0.]])

Here is how we would use the wedge operator to take the same cross product as before:

In [ ]:
print(wedge_numpy(w_inB) @ v_inB)

### Taking cross products with sympy

Suppose we want to take the cross product
$ w \times v $
where $w$ and $v$ are described in coordinates as follows:

$$w^B = \begin{bmatrix} a \\ b \\ c \end{bmatrix} \qquad\qquad v^B = \begin{bmatrix} d \\ e \\ f \end{bmatrix}$$

First, create symbolic variables.

In [ ]:
a, b, c, d, e, f = sym.symbols('a, b, c, d, e, f')

Second, create $w$ and $v$ as sympy matrices:

In [ ]:
w_inB = sym.Matrix([a, b, c])
v_inB = sym.Matrix([d, e, f])

Now, the easiest way to take the cross product is with [sympy.cross](https://docs.sympy.org/latest/modules/matrices/matrices.html#sympy.matrices.matrices.MatrixBase.cross):

In [ ]:
w_inB.cross(v_inB)

An alternative is to use the wedge operator. Here is a function that implements this operator:

In [ ]:
def wedge_sympy(v):
    """
    Returns a skew-symmetric matrix M that can be used to implement
    the cross product "v x w" as the matrix product "M @ w" - assumes
    that v has shape (3, )
    """
    return sym.Matrix([[0., -v[2], v[1]], [v[2], 0., -v[0]], [-v[1], v[0], 0.]])

Here is how we would use the wedge operator to take the same cross product as before:

In [ ]:
wedge_sympy(w_inB) * v_inB

Let's plug in the same values we used for the numpy example, to confirm we get the same result:

In [ ]:
w_inB.cross(v_inB).subs([(a, 1), (b, 2), (c, 3), (d, 4), (e, 5), (f, 6)])

# 3. Derive relationship between motor power commands and forces and torques

Define parameters as symbolic variables.

In [ ]:
k_F, k_M, l = sym.symbols('k_F, k_M, l')

Define motor power commands as symbolic variables.

In [ ]:
m_1, m_2, m_3, m_4 = sym.symbols('m_1, m_2, m_3, m_4')

Find net force and torque from the first motor.

In [ ]:
# Net force
f_inB_of1 = sym.Matrix([0, 0, k_F * m_1])

# Point at which force is applied
p_inB_of1 = sym.Matrix([l, -l, 0])

# Net torque
tau_inB_of1 = sym.Matrix([0, 0, -k_M * m_1]) + p_inB_of1.cross(f_inB_of1)

Show the result.

In [ ]:
print('net force from rotor 1 (in the coordinates of frame B):')
display(f_inB_of1)
print('net torque from rotor 1  (in the coordinates of frame B):')
display(tau_inB_of1)

Find net force and torque from all other motors.

In [ ]:
# Motor 2
f_inB_of2 = sym.Matrix([0, 0, k_F * m_2])
p_inB_of2 = sym.Matrix([-l, -l, 0])
tau_inB_of2 = sym.Matrix([0, 0, k_M * m_2]) + p_inB_of2.cross(f_inB_of2)

# Motor 3
f_inB_of3 = sym.Matrix([0, 0, k_F * m_3])
p_inB_of3 = sym.Matrix([-l, l, 0])
tau_inB_of3 = sym.Matrix([0, 0, -k_M * m_3]) + p_inB_of3.cross(f_inB_of3)

# Motor 4
f_inB_of4 = sym.Matrix([0, 0, k_F * m_4])
p_inB_of4 = sym.Matrix([l, l, 0])
tau_inB_of4 = sym.Matrix([0, 0, k_M * m_4]) + p_inB_of4.cross(f_inB_of4)

Find total force and torque from all motors.

In [ ]:
f_inB_ofRotors = f_inB_of1 + f_inB_of2 + f_inB_of3 + f_inB_of4
tau_inB_ofRotors = tau_inB_of1 + tau_inB_of2 + tau_inB_of3 + tau_inB_of4

Show the result.

In [ ]:
print('net force from all rotors (in the coordinates of frame B):')
display(f_inB_ofRotors)
print('net torque from all rotors  (in the coordinates of frame B):')
display(tau_inB_ofRotors)

Define the matrix $P$ that maps motor power commands to net forces and torques:

$$
\begin{bmatrix} \tau_x \\ \tau_y \\ \tau_z \\ f_z \end{bmatrix}
=
P
\begin{bmatrix} m_1 \\ m_2 \\ m_3 \\ m_4 \end{bmatrix}
$$

In [ ]:
P = sym.Matrix([
    [tau_inB_ofRotors[0].coeff(a) for a in [m_1, m_2, m_3, m_4]],
    [tau_inB_ofRotors[1].coeff(a) for a in [m_1, m_2, m_3, m_4]],
    [tau_inB_ofRotors[2].coeff(a) for a in [m_1, m_2, m_3, m_4]],
    [f_inB_ofRotors[2].coeff(a) for a in [m_1, m_2, m_3, m_4]],
])

Show the result.

In [ ]:
P

Find the matrix $P^{-1}$ that maps forces and torques to rotor speeds:

$$
\begin{bmatrix} m_1 \\ m_2 \\ m_3 \\ m_4 \end{bmatrix}
=
P^{-1}
\begin{bmatrix} \tau_x \\ \tau_y \\ \tau_z \\ f_z \end{bmatrix}
$$

In [ ]:
Pinv = P.inv()

Show the result.

In [ ]:
Pinv

# 4. Examples

Define symbolic variables for mass and gravity.

In [ ]:
m, g = sym.symbols('m, g')

Compute the motor power commands required to achieve a given net torque and net force.

In [ ]:
Pinv * sym.Matrix([0,
                   0,
                   0,
                   0])